<a href="https://colab.research.google.com/github/chandrakiranck/fuzzy-adventure/blob/master/rf_optimal_threshold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,confusion_matrix,roc_curve


In [0]:
data=pd.read_csv("/content/drive/My Drive/casestudy datasets/credit-card-default.csv")
data.head()

In [0]:
data=data.drop(["ID"],axis=1)
cat_cols=["SEX","EDUCATION","MARRIAGE"]
data=pd.get_dummies(data,columns=cat_cols,drop_first=True)

In [0]:
data

In [0]:
y=data["defaulted"]
X=data.drop(["defaulted"],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

In [0]:
Best_Params=  {'n_estimators': 200, 'min_samples_split': 60, 'min_samples_leaf': 100, 'max_depth': 10}

In [0]:
rf=RandomForestClassifier(**Best_Params)
rf.fit(X_train,y_train)

In [0]:
y_train_pred=rf.predict(X_train)
y_train_prob=rf.predict_proba(X_train)[:,1]
print(confusion_matrix(y_train,y_train_pred))
print("auc score of train data",roc_auc_score(y_train,y_train_prob))

In [0]:
fpr,tpr,thresholds=roc_curve(y_train,y_train_prob)
plt.plot(fpr,tpr)
plt.plot(fpr,fpr)
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC_AUC CURVE")

If we assume that cost of wrong pediction of defaulters is same as cost of wrong prediction of non-defaulters.
Then optimal cut-off is where the difference of fpr and tpr is maximum

In [0]:
fig,ax1=plt.subplots()
ax1.plot(fpr,tpr-fpr)

ax2=ax1.twinx()
ax2.plot(fpr,thresholds,"r")


OPTIMAL THRESHOLD

In [0]:
optimal_idx=np.argmax(tpr-fpr)
optimal_threshold=thresholds[optimal_idx]
optimal_threshold

In [0]:
dfts=pd.read_csv("/content/drive/My Drive/casestudy datasets/credit-card-default_toscore.csv")

In [0]:
dfts.head()

In [0]:
data=pd.read_csv("/content/drive/My Drive/casestudy datasets/credit-card-default.csv")
data=data.drop(["defaulted"],axis=1)
data.shape

In [0]:
dfall=pd.concat((data,dfts),axis=0)
cat_cols=["SEX","EDUCATION","MARRIAGE"]
dfall=pd.get_dummies(dfall,columns=cat_cols,drop_first=True)
dfts=dfall.iloc[30000:,]

In [0]:
ID=dfts[["ID"]]
dfts=dfts.drop(["ID"],axis=1)

In [0]:
dfts_prob=rf.predict_proba(dfts)[:,1]

In [0]:
ID["prob"]=dfts_prob

In [0]:
ID["prob"]

In [0]:
ID=ID[ID["prob"]>0.2316423943040094]

In [0]:
ID

In most situations cost of a defaulter is much larger than the benefit of a  non-defaulter .


Hence the previous assumption is weak.


in the next exercise we will consider  the cost and benfit of all the possible cases

In [0]:
max(y_train_prob)


In [0]:
gain=[]
i_vals=[]


for i in np.linspace(0,0.8,50):
  rf.fit(X_train,y_train)
  y_prob=rf.predict_proba(X_train)
  y_prob=pd.DataFrame(y_prob)
  y_pred=y_prob[1].apply(lambda x:1 if x>i else 0)
  conf=confusion_matrix(y_train,y_pred)

  total_benfit=conf[0][0]*75-conf[0][1]*75-conf[1][0]*200+conf[1][1]*200
  gain.append(total_benfit)
  i_vals.append(i)

print("Max gain :  ",max(gain))

In [0]:
plt.plot(i_vals,gain)